In [ ]:
# Default locations if parameter not passed
# benchmark_data_path = "../../.././bench_metadata.pkl"
benchmark_data_path = "/home/users/r176848/remy/spinner/bench_metadata.pkl"
output_folder = "./output"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from rich import print as rprint
import seaborn as sns
import os
from scipy.stats import bootstrap

benchmark_data = pickle.load(open(benchmark_data_path, "rb"))

metadata = benchmark_data["metadata"]
df = pd.DataFrame(benchmark_data["dataframe"])

# create the output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


rprint(df.columns)
rprint(df.head())
rprint(df.tail())

In [ ]:
sns.set_theme()

colors = [
    "#7f7f7f",  # grey
    "#621dac",  # main purple
    "#c5702d",  # orange
    "#000000",  # black,
    "#099892",  # teal
    "#ffd400",  # yellow
    "#7e57c4",  # pink/purple,
]
colors_backup = colors.copy()

In [ ]:
def calculate_bootstrap_error(data):
    if metadata["runs"] < 2:
        return 0
    res = bootstrap(
        (data,), np.mean, confidence_level=0.95, n_resamples=1000, method="basic"
    )
    return res.standard_error

In [ ]:
bench_names = df["name"].unique()

for bench in bench_names:
    for plot_setup in metadata[bench]["plot_axis"]:
        bench_df = df[df["name"] == bench].copy()

        y_ax = plot_setup["y_axis"]
        x_ax = plot_setup["x_axis"]
        group_by = plot_setup["group_by"]
        title = plot_setup["title"]
        n_runs = metadata["runs"]

        # make sure Y axis is numeric
        bench_df[y_ax] = pd.to_numeric(bench_df[y_ax])
        columns_with_nan = bench_df.columns[bench_df.isna().any()].tolist()

        # Drop columns with NaN
        bench_df = bench_df.drop(columns=columns_with_nan)
        bench_df = bench_df.loc[:, ~bench_df.columns.duplicated()]

        # Group by and calculate mean and bootstrap error
        grouped = (
            bench_df.groupby(group_by)[y_ax]
            .agg(["mean", calculate_bootstrap_error])
            .reset_index()
        )

        rprint(grouped)
        # plt.figure(figsize=(3.3, 2.4))

        plt.errorbar(
            grouped[x_ax],
            grouped["mean"],
            yerr=grouped["calculate_bootstrap_error"],
            fmt="o-",
            capsize=5,
            label=f"{bench}-{y_ax}",
        )

        plt.xlabel(x_ax)
        plt.xticks(grouped[x_ax])
        plt.ylabel(y_ax)
        plt.grid(True)
        plt.legend()

        plt.title(f"{title} | {n_runs} runs")
        plt.savefig(f"{output_folder}/{bench}_{y_ax}_{x_ax}.pdf")

    # plt.tight_layout()
    plt.savefig(f"{output_folder}/{bench}_{y_ax}_{x_ax}-GROUPED.pdf")
    plt.show()